Simone tries to be usefull and fail part 546


In [1]:
import pandas as pd
import numpy as np
import PyIRI
import PyIRI.edp_update as ml
import PyIRI.plotting as plot

# --- Define event start and end ---
start_date_string = '2018-04-20 21:00:00'
#end_date_string = '2018-04-21 03:00:00'
#'2024-06-29T18:00:00'  # Example: spans to next day

start_date = pd.to_datetime(start_date_string)

#make the end the start dat + 12 hours
end_date_string = (start_date + pd.Timedelta(hours=12)).strftime('%Y-%m-%d %H:%M:%S')

end_date = pd.to_datetime(end_date_string)

print(f"Event from {start_date} to {end_date}")

# --- Load F10.7 index ---
indices_F10 = pd.read_csv('/mnt/ionosphere-data/solar_env_tech_indices/Indices_F10_processed.csv')
indices_F10['Datetime'] = pd.to_datetime(indices_F10['Datetime'])

# Filter for the event date range (normalized to days)
dates_in_event = pd.date_range(start=start_date.normalize(), end=end_date.normalize(), freq='D')

# Build list of day-wise tuples for IRI input
iri_runs = []
for date in dates_in_event:
    day_start = max(start_date, date)
    day_end = min(end_date, date + pd.Timedelta(days=1) - pd.Timedelta(seconds=1))

    hr_res = 0.25 # 15 minute cadence generation

    # Create hourly range within the current day
    ahr = np.arange(day_start.hour, day_end.hour + hr_res, hr_res)

    # Get F10.7 value for this date (or fallback)
    f10_row = indices_F10[indices_F10['Datetime'] == date]
    f107 = f10_row['F10'].values[0] if not f10_row.empty else 150.0

    iri_runs.append({
        "year": date.year,
        "month": date.month,
        "day": date.day,
        "ahr": ahr,
        "f107": f107
    })

# --- Spatial and altitude grids ---
dlon = 1
dlat = 1
alon_2d, alat_2d = np.mgrid[-180:180:dlon, -90:90:dlat]
alon = np.reshape(alon_2d, alon_2d.size)
alat = np.reshape(alat_2d, alat_2d.size)

alt_res = 720
alt_min = 0
alt_max = 36000
aalt = np.arange(alt_min, alt_max, alt_res)

ccir_or_ursi = 1  # IRI option

# --- Run pyIRI for each day/hour range ---
all_edp = []
for run in iri_runs:
    print(f"Running IRI for {run['year']}-{run['month']:02}-{run['day']:02} with F10.7={run['f107']} hours={run['ahr']}")

    f2, f1, e_peak, es_peak, sun, mag, edp = ml.IRI_density_1day(
        run["year"], run["month"], run["day"],
        run["ahr"], alon, alat, aalt,
        run["f107"], PyIRI.coeff_dir, ccir_or_ursi
    )
    all_edp.append(edp)


# You can now concatenate or analyze the EDP results in `all_edp`


Event from 2018-04-20 21:00:00 to 2018-04-21 09:00:00
Running IRI for 2018-04-20 with F10.7=73.0 hours=[21.   21.25 21.5  21.75 22.   22.25 22.5  22.75 23.  ]
Running IRI for 2018-04-21 with F10.7=76.8 hours=[0.   0.25 0.5  0.75 1.   1.25 1.5  1.75 2.   2.25 2.5  2.75 3.   3.25
 3.5  3.75 4.   4.25 4.5  4.75 5.   5.25 5.5  5.75 6.   6.25 6.5  6.75
 7.   7.25 7.5  7.75 8.   8.25 8.5  8.75 9.  ]


In [2]:
# --- Compute TEC for each EDP in all_edp ---
all_tec = []
for i, edp in enumerate(all_edp):
    tec = PyIRI.main_library.edp_to_vtec(edp, aalt, min_alt=0.0, max_alt=36000.)
    all_tec.append(tec)
    print(f"TEC computed for day {i + 1}, shape: {tec.shape}")

TEC computed for day 1, shape: (9, 64800)
TEC computed for day 2, shape: (37, 64800)


In [ ]:
def get_tec_for_datetime(target_dt, iri_runs, all_tec, alon_2d):
    for i, run in enumerate(iri_runs):
        date = pd.Timestamp(run["year"], run["month"], run["day"])
        ahr = run["ahr"]
        start_hr = date + pd.to_timedelta(ahr[0], unit="h")
        end_hr = date + pd.to_timedelta(ahr[-1], unit="h")

        if start_hr <= target_dt <= end_hr:
            # Convert target time to fractional hour
            target_hr = (target_dt - date).total_seconds() / 3600.0
            hour_idx = np.argmin(np.abs(ahr - target_hr))

            tec_flat = all_tec[i][hour_idx]
            tec_2d = tec_flat.reshape(alon_2d.shape)
            return np.rot90(tec_2d), target_hr

    raise ValueError("Target datetime not in any IRI time range.")


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np


target_dt = start_date#pd.Timestamp("2024-06-29T00:00:00")
tec_2d, matched_hr = get_tec_for_datetime(target_dt, iri_runs, all_tec, alon_2d)

print(f"TEC for {target_dt} (closest match: {matched_hr:.2f} UTC)")

# Now plot `tec_2d` as usual

# Plotting
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

im = ax.imshow(
    tec_2d,
    extent=[-180, 180, -90, 90],
    origin='upper',  # Use 'lower' so that latitudes are plotted correctly
    cmap='jet',
    vmin=0,
    vmax=np.percentile(tec_2d, 95),
    transform=ccrs.PlateCarree()
)

# Add geographic features
ax.coastlines(color='white', linewidth=0.5)
gl = ax.gridlines(draw_labels=True, alpha=0.3)
gl.top_labels = False
gl.right_labels = False

# Colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.6, pad=0.02)
cbar.set_label('VTEC (TECU)', fontsize=12)

plt.title("Vertical TEC", fontsize=14)
plt.show()


In [ ]:
import os
import gzip
import io
from netCDF4 import Dataset
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

def convert_to_doy(date_obj):
    """Convert datetime object to zero-padded day of year string (e.g., '001', '059')."""
    doy = date_obj.timetuple().tm_yday
    return f"{doy:03d}"

def list_jpld_files(data_dir, start_date, end_date):
    """
    List JPLD files in folder matching the date range.
    Filenames like: 'jpld{DOY}0.{YYYY}i.nc.gz'
    """
    file_list = []
    date_range = pd.date_range(start=start_date.normalize(), end=end_date.normalize(), freq='D')
    year_str = str(start_date.year)[2:]
    
    for date in date_range:
        doy_str = convert_to_doy(date)
        print(f"Checking for file: jpld{doy_str}0.{year_str}i.nc.gz")
        filename = f"jpld{doy_str}0.{year_str}i.nc.gz"
        filepath = os.path.join(data_dir, filename)
        if os.path.isfile(filepath):
            file_list.append(filepath)
        else:
            print(f"Warning: File not found {filepath}")
    return file_list

def load_jpld_data(files, target_start_dt, target_end_dt):
    """
    Load and extract TEC maps for times within [target_start_dt, target_end_dt].
    Returns:
        - List of TEC arrays with shape (time, lat, lon)
        - latitudes, longitudes arrays
        - list of datetime objects matching each time slice
    """
    all_tec = []
    all_times = []
    lat, lon = None, None
    
    j2000 = datetime(2000, 1, 1, 12, 0, 0)
    
    for fpath in files:
        with gzip.open(fpath, 'rb') as f:
            decompressed_data = f.read()
        nc_buffer = io.BytesIO(decompressed_data)
        with Dataset('in_memory.nc', mode='r', memory=nc_buffer.read()) as nc:
            time_seconds = nc.variables['time'][:]
            times = [j2000 + timedelta(seconds=float(t)) for t in time_seconds]
            
            # Select indices within target time interval
            indices = [i for i, t in enumerate(times) if target_start_dt <= t <= target_end_dt]
            
            if len(indices) == 0:
                continue  # no relevant data in this file
            
            tecmap = nc.variables['tecmap'][:]  # shape (time, lat, lon)
            if lat is None or lon is None:
                lat = nc.variables['lat'][:]
                lon = nc.variables['lon'][:]
            
            # Extract only the relevant time slices
            tec_subset = tecmap[indices, :, :]
            all_tec.append(tec_subset)
            all_times.extend([times[i] for i in indices])
    
    if len(all_tec) == 0:
        raise RuntimeError("No TEC data found for given date/time range.")
    
    # Concatenate all days along the time dimension
    all_tec = np.concatenate(all_tec, axis=0)
    
    return all_tec, lat, lon, all_times

In [ ]:
# Define your event start and end times (matching your IRI inputs)
# start_date = pd.to_datetime('2024-06-28T09:00:00')
# end_date = pd.to_datetime('2024-06-29T18:00:00')

# Folder where JPLD files are stored (update accordingly)
jpld_dir_path = f'/mnt/ionosphere-data/jpld/raw/{start_date.year}/'

# Step 1: List relevant files
jpld_files = list_jpld_files(jpld_dir_path, start_date, end_date)

# Step 2: Load data and extract time slices matching event time range
tec_data, lat, lon, tec_times = load_jpld_data(jpld_files, start_date, end_date)

print(f"Loaded TEC data shape: {tec_data.shape}")
print(f"Number of time steps: {len(tec_times)}")

# Now you have:
# tec_data: array of shape (time, lat, lon), 15-minute cadence (usually 96 per day)
# lat, lon: coordinate arrays
# tec_times: datetime list for each time index

# Optional: flip latitudes if needed for consistency with IRI data
tec_data = np.flip(tec_data, axis=1)  # flips lat axis

# Step 3: Create a structure similar to IRI outputs
jpld_struct = {
    "tec": tec_data,
    "lat": lat,
    "lon": lon,
    "times": tec_times
}


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

def plot_tec_map(time_idx, data_struct):
    tec_map = data_struct["tec"][time_idx, :, :]
    lat = data_struct["lat"]
    lon = data_struct["lon"]
    dt = data_struct["times"][time_idx]
    
    fig = plt.figure(figsize=(12, 6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    im = ax.pcolormesh(lon, lat, np.log10(tec_map), cmap='jet', vmin=0, vmax=1.5, shading='auto')
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.3)
    ax.set_title(f'JPLD GNSS GIM TEC Map on {dt.strftime("%Y-%m-%d %H:%M UTC")}', fontsize=14)
    plt.colorbar(im, ax=ax, label='log10(TEC)')
    plt.show()

# Plot first time slice
plot_tec_map(0, jpld_struct)


In [ ]:
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

iri_times = []
for run in iri_runs:
    base_date = datetime(run["year"], run["month"], run["day"])
    for hour in run["ahr"]:
        iri_times.append(base_date + timedelta(hours=float(hour)))


matched_indices = []
for iri_time in iri_times:
    time_diffs = np.array([abs((jpld_t - iri_time).total_seconds()) for jpld_t in jpld_struct['times']])
    closest_idx = time_diffs.argmin()
    matched_indices.append(closest_idx)


def plot_side_by_side(iri_tec, jpld_tec, lat, lon, dt):
    fig, axs = plt.subplots(1, 2, figsize=(16, 6), subplot_kw={'projection': ccrs.PlateCarree()})

    # Plot IRI TEC
    ax = axs[0]
    im = ax.imshow(
        iri_tec, extent=[-180, 180, -90, 90], origin='upper', cmap='jet',
        vmin=0, vmax=np.percentile(iri_tec, 95), transform=ccrs.PlateCarree()
    )
    ax.coastlines()
    ax.set_title(f'IRI TEC {dt.strftime("%Y-%m-%d %H:%M UTC")}')
    plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)

    # Plot JPLD TEC
    ax = axs[1]
    im = ax.pcolormesh(lon, lat, jpld_tec, cmap='jet', vmin=0, vmax=np.percentile(iri_tec, 95), shading='auto')
    ax.coastlines()
    ax.set_title(f'JPLD TEC {dt.strftime("%Y-%m-%d %H:%M UTC")}')
    plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()

# Example usage for the first IRI time
i = 0
iri_tec_map = all_tec[i][np.argmin(np.abs(iri_runs[i]["ahr"] - (iri_times[i]-datetime(iri_runs[i]["year"], iri_runs[i]["month"], iri_runs[i]["day"])).total_seconds()/3600))]
iri_tec_map = np.rot90(iri_tec_map.reshape(alon_2d.shape))

jpld_idx = matched_indices[i]
jpld_tec_map = jpld_struct["tec"][jpld_idx, :, :]

plot_side_by_side(iri_tec_map, jpld_tec_map, jpld_struct["lat"], jpld_struct["lon"], iri_times[i])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Flatten the TEC maps into 1D arrays
tec_iri_flat = iri_tec_map.flatten()
tec_jpld_flat = jpld_tec_map.flatten()

# Mask NaNs or negative values if needed
mask = np.isfinite(tec_iri_flat) & np.isfinite(tec_jpld_flat) & (tec_iri_flat > 0) & (tec_jpld_flat > 0)
tec_iri_clean = tec_iri_flat[mask]
tec_jpld_clean = tec_jpld_flat[mask]

# Create 2D histogram
plt.figure(figsize=(8, 6))
hb = plt.hist2d(
    tec_iri_clean,
    tec_jpld_clean,
    bins=50,
    norm=plt.cm.colors.LogNorm(),  # Log scale to emphasize spread
    cmap='viridis'
)

# Add colorbar and labels
plt.colorbar(hb[3], label='Counts (log scale)')
plt.xlabel('IRI TEC (TECU)')
plt.ylabel('JPLD TEC (TECU)')
plt.title('2D Histogram: IRI vs JPLD TEC')

# Optional: add 1:1 line for reference
min_val = min(tec_iri_clean.min(), tec_jpld_clean.min())
max_val = max(tec_iri_clean.max(), tec_jpld_clean.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='1:1 line')
plt.legend()

plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()
